### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

/Users/artemzraev/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
X_test[['description', 'company_profile', 'benefits']].head(3)

,description,company_profile,benefits
0,Stylect is a dynamic startup that helps helps ...,NaN,We are negotiable on salary and there is the p...
1,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,NaN


In [11]:
import urllib.request
import json      


def get_prediction(x):
    description, company_profile, benefits = x
    body = {'description': description, 
            'company_profile': company_profile,
            'benefits': benefits} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [12]:
#get_prediction(1,10,1,1)

In [14]:
%%time
predictions = X_test[['description', 'company_profile', 
                      'benefits']].iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 315 ms, sys: 56.6 ms, total: 372 ms
Wall time: 5.46 s


In [15]:
predictions

0      0.026174
1      0.044576
2      0.004804
3      0.001057
4      0.001229
         ...   
495    0.008572
496    0.001839
497    0.620032
498    0.001418
499    0.001099
Length: 500, dtype: float64

In [7]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [10]:
get_prediction(("", None, None))

0.13518841848029955

In [9]:
get_prediction(("", "", ""))

0.13518841848029955